In [19]:
import numpy as np
from scipy import linalg
K1 = np.array([[2.,-1.,0,],
               [-1.,2.,-1.],
               [0.,-1.,1.]])

K2 = np.array([[1.,-1.,0,],
               [-1.,2.,-1.],
               [0.,-1.,2.]])

M1 = 3*np.array([[1.,0.,0,],
               [0.,1.,0.],
               [0.,0.,1.]])

M2 = 3*np.array([[1.,0.,0,],
               [0.,1.,0.],
               [0.,0.,1.]])


B12 = np.array([0.,0,1.0])
B21 = np.array([-1.,0,0.0])


def local2block(matrix_list):
    
    return linalg.block_diag(*matrix_list)

def local2hblock(matrix_list):
    
    return np.hstack(matrix_list)

penalty_parameter = 1.0E-0

def nonlinear(delta_u):
    if delta_u <0.0:
        return  -penalty_parameter*delta_u
    else:
        return 0.0
    
def deriv_nonlinear(delta_u):
    if delta_u <0.0:
        return  -penalty_parameter
    else:
        return 0.0

alpha = 0.0001
beta = 0.0000001



In [20]:
K = local2block([K1,K2])
M = local2block([M1,M2])
B = local2hblock([B12,B21])

In [21]:
from contpy import optimize as copt, frequency